__<font size=5><center>Projeto Integrador - Sistema de Clusterização de Clientes para CRM</center></font>__

__Author__: Rui Ferreira Póvoa

__School__: Digital House Brazil

__Class__: Data Science

__Due Date__:   29/11/2019

<a name="1"></a>
# 1. Data Loading

<a name="1.1"></a>
## 1.1. Importação de Bibliotecas

In [1]:
#bibliotecas para análise de dados:
import pandas as pd                                        # tables and data manipulations
import numpy as np                                         # vectors and matrices

In [2]:
#bibliotecas para ler o arquivo e realizar análises de tipo de csv
import csv
import itertools                                           # funções úteis

In [3]:
#import of heap algoryth for sorting
import heapq

In [4]:
#biblioteca de tempo
from datetime import date, datetime, timedelta

today_date = date.today()

## 1.2. Carregando os arquivos

In [5]:
filepath = "data\Produtos.csv"

In [6]:
#Trazendo o cabeçalho:
with open(filepath, 'r', encoding = 'utf8') as csvfile:
    #head = ''.join(itertools.islice(csvfile, 1))
    #print(head)
    reader = csv.reader(csvfile)
    lst_header = next(reader)
    print(lst_header)

['id_pedido', 'id_carrinho', 'status', 'desc_status', 'id_cliente', 'id_distribuidor', 'id_sku', 'id_produto', 'produto', 'anomes_carrinho', 'data_carrinho', 'anomes_status_pedido', 'data_status_pedido', 'quantidade', 'volumepedidos', 'volumecompras', 'volumependente', 'volumecanceladorecusado', 'meio_pagamento']


In [7]:
#preparando os dados para chamar no pandas
#se vai incluir NaN não pode ser usado o "int" com i tem de ser capitalizado "I"
dtype_list = [np.int32,np.int32,np.int16,'str',np.int32,
              np.int32,'Int32','Int32','str',np.int16,
              'str','Int32','str', np.float32,np.float32,
              np.float32,np.float32,np.float32, 'str']

dtype_dict = dict(zip(lst_header, dtype_list))
print(dtype_dict)

#passando datas
parse_dates = ['data_carrinho', 'data_status_pedido']

{'id_pedido': <class 'numpy.int32'>, 'id_carrinho': <class 'numpy.int32'>, 'status': <class 'numpy.int16'>, 'desc_status': 'str', 'id_cliente': <class 'numpy.int32'>, 'id_distribuidor': <class 'numpy.int32'>, 'id_sku': 'Int32', 'id_produto': 'Int32', 'produto': 'str', 'anomes_carrinho': <class 'numpy.int16'>, 'data_carrinho': 'str', 'anomes_status_pedido': 'Int32', 'data_status_pedido': 'str', 'quantidade': <class 'numpy.float32'>, 'volumepedidos': <class 'numpy.float32'>, 'volumecompras': <class 'numpy.float32'>, 'volumependente': <class 'numpy.float32'>, 'volumecanceladorecusado': <class 'numpy.float32'>, 'meio_pagamento': 'str'}


In [8]:
#deu erro a tenar chamar os resultados com os dtypes
df = pd.read_csv(filepath,
                 sep=',',
                 header=0,
                 encoding="utf8",
                 dtype = dtype_dict,
                 parse_dates=parse_dates)

In [10]:
filepath = "data\Pedidos.csv"
dtype_list = [np.int32,np.int32,np.int16,'str',np.int32,
              np.int32,np.int16,'str',np.int16,'str',
              np.int32,np.float32,np.float32,np.float32,np.float32]
dtype_dict = dict(zip(lst_header, dtype_list))
parse_dates = ['data_carrinho', 'data_status_pedido']

df_pedidos = pd.read_csv(filepath,
                 sep=',',
                 header=0,
                 encoding="utf8",
                 dtype = dtype_dict,
                 parse_dates=parse_dates)

# 2. Data Pre-processing

## 2.1 Dimensionalidade do DataFrame

In [12]:
df.shape

(298702, 19)

In [13]:
df_pedidos.shape

(78817, 15)

## 2.2. Visualização primária dos dados

In [14]:
df.head(5)

,id_pedido,id_carrinho,status,desc_status,id_cliente,id_distribuidor,id_sku,id_produto,produto,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,quantidade,volumepedidos,volumecompras,volumependente,volumecanceladorecusado,meio_pagamento
0,107783,71935,0,Carrinho Abandonado,30696,137,4696,16544,PAPEL TOALHA INTERFOLHA 2 DOBRAS SOFT BRANCO P...,5204,2018-12-11 12:05:40.081189+00:00,NaN,NaT,4.0,59.959999,NaN,NaN,NaN,DEPÓSITO EM CONTA ANTECIPADO
1,3,3,0,Carrinho Abandonado,3,15,NaN,NaN,- BALIX,4903,2015-11-11 20:28:39.699416+00:00,NaN,NaT,1.0,22.900000,NaN,NaN,NaN,DESCONTINUADO
2,53780,33739,0,Carrinho Abandonado,3,4258,29813,1657959,CÁPSULAS DE CAFÉ ESPRESSO FORTISSIMO PARA MÁQU...,5196,2018-04-18 15:01:30.063939+00:00,NaN,NaT,1.0,14.500000,NaN,NaN,NaN,DESCONTINUADO
3,53782,33737,0,Carrinho Abandonado,4583,2900,18244,637259,ESCOVA DENTAL MACIA ESSENCIAL CLEAN UNIDADE -...,5196,2018-04-18 14:48:41.568361+00:00,NaN,NaT,1.0,1.690000,NaN,NaN,NaN,DESCONTINUADO
4,53773,33739,0,Carrinho Abandonado,3,1203,13021,61708,MACARRÃO ESPAGUETE (8) COM OVOS PACOTE 500G - ...,5196,2018-04-18 15:01:30.063939+00:00,NaN,NaT,24.0,45.119999,NaN,NaN,NaN,DESCONTINUADO


In [15]:
df.dtypes.value_counts()

float32                5
int32                  4
object                 3
Int32                  3
int16                  2
datetime64[ns, UTC]    2
dtype: int64

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298702 entries, 0 to 298701
Data columns (total 19 columns):
id_pedido                  298702 non-null int32
id_carrinho                298702 non-null int32
status                     298702 non-null int16
desc_status                298702 non-null object
id_cliente                 298702 non-null int32
id_distribuidor            298702 non-null int32
id_sku                     231509 non-null Int32
id_produto                 270994 non-null Int32
produto                    298702 non-null object
anomes_carrinho            298702 non-null int16
data_carrinho              298702 non-null datetime64[ns, UTC]
anomes_status_pedido       52322 non-null Int32
data_status_pedido         52322 non-null datetime64[ns, UTC]
quantidade                 298536 non-null float32
volumepedidos              298401 non-null float32
volumecompras              31799 non-null float32
volumependente             827 non-null float32
volumecanceladorecusado 

Conclui que apesar de ter 298702 carrinhos criados só 52322 viram pedidos

## 2.4. Dados em Falta (NaN)

In [17]:
def missing_data(data):
    #transformando os dados num objecto Series
    total = data.isnull().sum()
    
    # calculando a percentagem por coluna
    percent = round(data.isnull().sum() / len(data) * 100, 2)
    
    #A função concat coloca uma Series/Dataframe a seguir ao outro segundo o eixo que se quiser
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    
    #colcando o data types das colunas numa lista
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [18]:
%%time
missing_data(df)

Wall time: 263 ms


,id_pedido,id_carrinho,status,desc_status,id_cliente,id_distribuidor,id_sku,id_produto,produto,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,quantidade,volumepedidos,volumecompras,volumependente,volumecanceladorecusado,meio_pagamento
Total,0,0,0,0,0,0,67193,27708,0,0,0,246380,246380,166,301,266903,297875,279048,22714
Percent,0,0,0,0,0,0,22.49,9.28,0,0,0,82.48,82.48,0.06,0.1,89.35,99.72,93.42,7.6
Types,int32,int32,int16,object,int32,int32,Int32,Int32,object,int16,"datetime64[ns, UTC]",Int32,"datetime64[ns, UTC]",float32,float32,float32,float32,float32,object


Como existem 67193 sku não encontrados, não se consegue de forma lógica analisar o produto e sua marca, pelo que se fará um drop dessas linhas

In [19]:
df.dropna(axis=0, subset=['id_sku'], inplace = True)

In [20]:
missing_data(df)

,id_pedido,id_carrinho,status,desc_status,id_cliente,id_distribuidor,id_sku,id_produto,produto,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,quantidade,volumepedidos,volumecompras,volumependente,volumecanceladorecusado,meio_pagamento
Total,0,0,0,0,0,0,0,0,0,0,0,185842,185842,0,0,203542,230715,214603,19338
Percent,0,0,0,0,0,0,0,0,0,0,0,80.27,80.27,0,0,87.92,99.66,92.7,8.35
Types,int32,int32,int16,object,int32,int32,Int32,Int32,object,int16,"datetime64[ns, UTC]",Int32,"datetime64[ns, UTC]",float32,float32,float32,float32,float32,object


In [21]:
df.shape

(231509, 19)

In [22]:
df.desc_status.value_counts()

Carrinho Abandonado    185842
Aprovado                27967
Cancelado/Recusado      16906
Pendente                  794
Name: desc_status, dtype: int64

# 3. Feature Engineering

In [40]:
def f(x):
    d = {}
    
    d['pedidos'] = x['id_pedido'].count()
    d['pedidos_efetivos'] = x['volumecompras'].count()
    d['pedidos_cancelados'] = x['volumecanceladorecusado'].count()
    d['volume_compras'] = x['volumecompras'].sum()
    d['volume_cancelados'] = x['volumecanceladorecusado'].sum()
    d['max_pedido'] = x['volumecanceladorecusado'].max()
    d['tk_med_pedido'] =  x['volumecompras'].sum() / x['volumecompras'].count()
    d['data_cadastro'] = x['data_carrinho'].min()
    d['data_ult_pedido'] = x['data_status_pedido'].max()
     
    d['frequencia'] = (x['data_status_pedido'] - x['data_status_pedido'].shift(1)).mean() / np.timedelta64(1, 'D')
    d['recencia_max'] = (x['data_status_pedido'] - x['data_status_pedido'].shift(1)).max() / np.timedelta64(1, 'D')
    
    return pd.Series(d, index=['pedidos', 'pedidos_efetivos', 
                               'pedidos_cancelados', 'volume_compras',
                               'volume_cancelados','max_pedido',
                               'tk_med_pedido', 'data_cadastro',
                               'data_ult_pedido','frequencia','recencia_max'])

df_pedidos.sort_values(by='data_status_pedido', ascending = True).groupby(by='id_cliente').apply(f).sort_values(by='pedidos_efetivos', ascending = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


,pedidos,pedidos_efetivos,pedidos_cancelados,volume_compras,volume_cancelados,tk_med_pedido,data_cadastro,data_ult_pedido,frequencia,recencia_max
id_cliente,,,,,,,,,,
3,11125,1299,46,648154.520829,22187.4104,498.964219,2015-11-11 20:28:39.699416+00:00,2019-10-28 15:22:54.006572+00:00,1.050678,55.016411
1072,2504,242,19,186223.390200,13828.3778,769.518141,2016-11-16 13:17:29.442304+00:00,2019-10-29 15:28:26.643484+00:00,4.137238,53.610441
1456,889,237,19,402749.971600,31949.1100,1699.366969,2017-01-31 16:31:12.816499+00:00,2019-10-24 13:01:55.891945+00:00,3.427059,56.178038
130,502,133,6,117172.042740,3599.0600,880.992803,2016-03-21 15:42:25.575691+00:00,2019-02-12 14:53:49.822570+00:00,7.666422,294.976786
4211,340,117,5,79424.871230,3699.0560,678.845053,2018-02-23 12:15:37.023686+00:00,2019-09-28 10:13:54.830582+00:00,4.809147,42.893191
...,...,...,...,...,...,...,...,...,...,...
69700,1,0,0,0.000000,0.0000,NaN,2019-02-25 19:35:19.358659+00:00,NaT,NaN,NaN
69707,2,0,0,0.000000,0.0000,NaN,2019-02-25 19:42:57.947170+00:00,NaT,NaN,NaN
69723,1,0,0,0.000000,0.0000,NaN,2019-02-25 20:06:45.955674+00:00,NaT,NaN,NaN


In [19]:
df_clients = df_clients.pedidos.loc[(df_clients.pedidos!=0)]

In [20]:
df_clients

id_cliente
3          8569
1072       1174
218003      954
4050        757
4211        747
           ... 
3978          1
457985        1
1418475       1
823           1
69431         1
Name: pedidos, Length: 1352, dtype: int64